In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /kaggle/working/processed_data.zip
100%|██████████████████████████████████████| 5.62M/5.62M [00:00<00:00, 31.2MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

# !pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 103.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 76.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 104.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00

In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [7]:


models_list_3=[
        # "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",
        "infgrad/jasper_en_vision_language_v1",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
# "FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",
        "infgrad/jasper_en_vision_language_v1",

# "bert",

]


In [8]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
# langs=["eng"]
langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [9]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [10]:
all_csv

,id,anger,disgust,fear,joy,sadness,lang,clean_message
0,afr_train_track_a_00001,0,0,0,0,0,afr,die grondeienaars het die departement genader ...
1,afr_train_track_a_00002,0,0,0,0,0,afr,dit is verder n erkende feit dat daar meningsv...
2,afr_train_track_a_00003,0,0,0,0,1,afr,baie families in die weskaap is in rou gedompe...
3,afr_train_track_a_00004,0,0,0,1,0,afr,ons wil u deelmaak van die werk wat ons doen
4,afr_train_track_a_00005,0,0,0,1,0,afr,en dit onderstreep waarom naln en nelm gesamen...
...,...,...,...,...,...,...,...,...
1217,afr_train_track_a_01218,1,1,0,0,0,afr,rassisme moet finaal vir eens en vir altyd beg...
1218,afr_train_track_a_01219,0,0,0,1,0,afr,sy werk hard en verwag vinnige resultate van h...
1219,afr_train_track_a_01220,0,0,0,1,0,afr,tweehonderd sesensestig kinders van al die pro...
1220,afr_train_track_a_01221,0,0,0,1,0,afr,onder andere het die weskaapse kultuurkommissi...


In [11]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,lang,clean_message


In [12]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [13]:
train_data=all_csv


In [14]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [15]:

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,lang,clean_message
0,afr_train_track_a_00001,0,0,0,0,0,afr,die grondeienaars het die departement genader ...
1,afr_train_track_a_00002,0,0,0,0,0,afr,dit is verder n erkende feit dat daar meningsv...
2,afr_train_track_a_00003,0,0,0,0,1,afr,baie families in die weskaap is in rou gedompe...
3,afr_train_track_a_00004,0,0,0,1,0,afr,ons wil u deelmaak van die werk wat ons doen
4,afr_train_track_a_00005,0,0,0,1,0,afr,en dit onderstreep waarom naln en nelm gesamen...
...,...,...,...,...,...,...,...,...
1217,afr_train_track_a_01218,1,1,0,0,0,afr,rassisme moet finaal vir eens en vir altyd beg...
1218,afr_train_track_a_01219,0,0,0,1,0,afr,sy werk hard en verwag vinnige resultate van h...
1219,afr_train_track_a_01220,0,0,0,1,0,afr,tweehonderd sesensestig kinders van al die pro...
1220,afr_train_track_a_01221,0,0,0,1,0,afr,onder andere het die weskaapse kultuurkommissi...


In [16]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,disgust,fear,joy,sadness,lang,clean_message
0,afr_train_track_a_00001,0,0,0,0,0,afr,die grondeienaars het die departement genader ...
1,afr_train_track_a_00002,0,0,0,0,0,afr,dit is verder n erkende feit dat daar meningsv...
2,afr_train_track_a_00003,0,0,0,0,1,afr,baie families in die weskaap is in rou gedompe...
3,afr_train_track_a_00004,0,0,0,1,0,afr,ons wil u deelmaak van die werk wat ons doen
4,afr_train_track_a_00005,0,0,0,1,0,afr,en dit onderstreep waarom naln en nelm gesamen...
...,...,...,...,...,...,...,...,...
1217,afr_train_track_a_01218,1,1,0,0,0,afr,rassisme moet finaal vir eens en vir altyd beg...
1218,afr_train_track_a_01219,0,0,0,1,0,afr,sy werk hard en verwag vinnige resultate van h...
1219,afr_train_track_a_01220,0,0,0,1,0,afr,tweehonderd sesensestig kinders van al die pro...
1220,afr_train_track_a_01221,0,0,0,1,0,afr,onder andere het die weskaapse kultuurkommissi...


In [17]:
train_data.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
lang             object
clean_message    object
dtype: object

In [18]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,lang,clean_message
0,0,afr_train_track_a_00001,0,0,0,0,0,afr,die grondeienaars het die departement genader ...
1,1,afr_train_track_a_00002,0,0,0,0,0,afr,dit is verder n erkende feit dat daar meningsv...
2,2,afr_train_track_a_00003,0,0,0,0,1,afr,baie families in die weskaap is in rou gedompe...
3,3,afr_train_track_a_00004,0,0,0,1,0,afr,ons wil u deelmaak van die werk wat ons doen
4,4,afr_train_track_a_00005,0,0,0,1,0,afr,en dit onderstreep waarom naln en nelm gesamen...
...,...,...,...,...,...,...,...,...,...
1217,1217,afr_train_track_a_01218,1,1,0,0,0,afr,rassisme moet finaal vir eens en vir altyd beg...
1218,1218,afr_train_track_a_01219,0,0,0,1,0,afr,sy werk hard en verwag vinnige resultate van h...
1219,1219,afr_train_track_a_01220,0,0,0,1,0,afr,tweehonderd sesensestig kinders van al die pro...
1220,1220,afr_train_track_a_01221,0,0,0,1,0,afr,onder andere het die weskaapse kultuurkommissi...


In [19]:
train_data

,id,anger,disgust,fear,joy,sadness,lang,clean_message
0,afr_train_track_a_00001,0,0,0,0,0,afr,die grondeienaars het die departement genader ...
1,afr_train_track_a_00002,0,0,0,0,0,afr,dit is verder n erkende feit dat daar meningsv...
2,afr_train_track_a_00003,0,0,0,0,1,afr,baie families in die weskaap is in rou gedompe...
3,afr_train_track_a_00004,0,0,0,1,0,afr,ons wil u deelmaak van die werk wat ons doen
4,afr_train_track_a_00005,0,0,0,1,0,afr,en dit onderstreep waarom naln en nelm gesamen...
...,...,...,...,...,...,...,...,...
1217,afr_train_track_a_01218,1,1,0,0,0,afr,rassisme moet finaal vir eens en vir altyd beg...
1218,afr_train_track_a_01219,0,0,0,1,0,afr,sy werk hard en verwag vinnige resultate van h...
1219,afr_train_track_a_01220,0,0,0,1,0,afr,tweehonderd sesensestig kinders van al die pro...
1220,afr_train_track_a_01221,0,0,0,1,0,afr,onder andere het die weskaapse kultuurkommissi...


In [20]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise'] #eng
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness'] #afr
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise'] #all


In [21]:
train_data['lang'].value_counts()

lang
afr    1222
Name: count, dtype: int64

In [22]:
sum(train_data['lang'].isna())

0

In [23]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,disgust,fear,joy,sadness,lang,clean_message


In [24]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,lang,clean_message


In [25]:
train_data[train_data["lang"]=="deu"].dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
lang             object
clean_message    object
dtype: object

In [26]:
nan_rows["lang"].value_counts()

Series([], Name: count, dtype: int64)

In [27]:
nan_rows.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
lang             object
clean_message    object
dtype: object

In [28]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

             id  disgust   fear    joy  sadness  lang  clean_message
lang                                                                
afr  0     True    False  False  False    False  True           True
     1     True    False  False  False    False  True           True
     2     True    False  False  False     True  True           True
     3     True    False  False   True    False  True           True
     4     True    False  False   True    False  True           True
...         ...      ...    ...    ...      ...   ...            ...
     1217  True     True  False  False    False  True           True
     1218  True    False  False   True    False  True           True
     1219  True    False  False   True    False  True           True
     1220  True    False  False   True    False  True           True
     1221  True    False  False   True    False  True           True

[1222 rows x 7 columns]


In [29]:
maxi=train_data['clean_message'].apply(len).max()
maxi

176

In [30]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,lang,clean_message
456,afr_train_track_a_00457,0,0,0,0,0,afr,die wysigingswetsontwerp op die provinsiale on...


In [31]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['die wysigingswetsontwerp op die provinsiale ontwikkelingsraad en die wetsontwerp op die weskaapse jeugkommissie is deur die kabinet onderteken vir voorlegging aan die parlement'],
      dtype=object)

In [32]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [33]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [34]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [35]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [36]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

         count_0  count_1  sum
anger        941       36  977
disgust      968        9  977
fear         879       98  977
joy          550      427  977
sadness      846      131  977


In [37]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [38]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([13.5694, 54.2778,  4.9847,  1.1440,  3.7290], device='cuda:0')

In [39]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [40]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [41]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [42]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [43]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [44]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [45]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [46]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [47]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [48]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [49]:
USE_LORA=False

In [50]:
!pip install -q sentence_transformers

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:00:00:01


In [51]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [52]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

infgrad/jasper_en_vision_language_v1


modules.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/246k [00:00<?, ?B/s]

custom_st.py:   0%|          | 0.00/3.12k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

In [53]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [54]:
# a=

In [55]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [56]:
device

'cuda'

In [57]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [58]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]
        # return self.tokenizer(
        #     self.texts[idx],
        #     truncation=True,
        #     padding="max_length",
        #     max_length=self.max_length,
        #     return_tensors="pt"
        # )


In [59]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [60]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)
x_valid_embeddings = extract_embeddings(x_valid, embedding_model, batch_size=32)

Extracting embeddings:   0%|          | 0/31 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:   3%|▎         | 1/31 [00:01<00:38,  1.29s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:   6%|▋         | 2/31 [00:02<00:30,  1.04s/it]

...

Extracting embeddings:  75%|███████▌  | 6/8 [00:04<00:01,  1.21it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:  88%|████████▊ | 7/8 [00:05<00:00,  1.20it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings: 100%|██████████| 8/8 [00:06<00:00,  1.25it/s]


In [61]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)
y_valid_flat = np.array(y_valid)

In [62]:
import xgboost as xgb

In [63]:
xgb_models = {}
predictions = {}

for i, label in enumerate(label_columns):
    print(f"Training XGBoost for label: {label}")
    # Use the computed class weights for the positive class
    scale_pos_weight = class_weights_tensor[i].item()
    # Initialize and train XGBoost classifier
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        tree_method='hist',
        scale_pos_weight=scale_pos_weight,  # Apply the class weight here
        device=device
    )
    xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
    xgb_models[label] = xgb_model

    # Predict on validation data
    predictions[label] = xgb_model.predict(x_valid_embeddings)

Training XGBoost for label: anger


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [06:29:20] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Training XGBoost for label: disgust
Training XGBoost for label: fear
Training XGBoost for label: joy
Training XGBoost for label: sadness


In [64]:
# Evaluate the models
results = []
for i, label in enumerate(label_columns):
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_valid_flat[:, i],
        predictions[label],
        average="binary"
    )
    accuracy = accuracy_score(y_valid_flat[:, i], predictions[label])
    results.append({
        "Label": label,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Accuracy": accuracy
    })

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
# Display evaluation results
results_df = pd.DataFrame(results)
print(results_df)

     Label  Precision    Recall  F1 Score  Accuracy
0    anger   0.000000  0.000000  0.000000  0.967347
1  disgust   0.000000  0.000000  0.000000  0.987755
2     fear   0.250000  0.043478  0.074074  0.897959
3      joy   0.627907  0.519231  0.568421  0.665306
4  sadness   0.500000  0.065217  0.115385  0.812245


In [66]:
print(f"{x} evaluates:")

for k, res in enumerate(results):

  print(f"Class '{res['Label']}': Precision: {res['Precision']:.4f}, Recall: {res['Recall']:.4f}, F1: {res['F1 Score']:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

infgrad/jasper_en_vision_language_v1 evaluates:
Class 'anger': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'disgust': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'fear': Precision: 0.2500, Recall: 0.0435, F1: 0.0741
Class 'joy': Precision: 0.6279, Recall: 0.5192, F1: 0.5684
Class 'sadness': Precision: 0.5000, Recall: 0.0652, F1: 0.1154
Overall accuracy: 0.8122
-------------------------------------------------------------------------



In [67]:
# Save the XGBoost models if needed
for label, model in xgb_models.items():
    model.save_model(f"xgb_model_{label}.json")

In [68]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()

    if len(lang_valid_data) == 0:
        continue

    # Extract embeddings for the current language's validation data
    lang_x_valid_embeddings = extract_embeddings(lang_x_valid, embedding_model)
    print()
    # Initialize metrics storage
    all_precisions, all_recalls, all_f1_scores, all_accuracies = [], [], [], []

    # Evaluate each label separately
    for i, label in enumerate(label_columns):
        # Predict using the corresponding XGBoost model
        lang_predictions = xgb_models[label].predict(lang_x_valid_embeddings)

        # Compute metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            np.array(lang_y_valid)[:, i],  # True labels for this label
            lang_predictions,
            average="binary",
            zero_division=0
        )
        accuracy = accuracy_score(np.array(lang_y_valid)[:, i], lang_predictions)

        # Store metrics
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1_scores.append(f1)
        all_accuracies.append(accuracy)

    # Aggregate metrics (average across all labels for the language)
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [np.mean(all_precisions)],
        "Recall": [np.mean(all_recalls)],
        "F1_score": [np.mean(all_f1_scores)],
        "Accuracy": [np.mean(all_accuracies)]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)


Extracting embeddings:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:  12%|█▎        | 1/8 [00:00<00:06,  1.12it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:  25%|██▌       | 2/8 [00:01<00:05,  1.17it/s]

...

Extracting embeddings:  88%|████████▊ | 7/8 [00:05<00:00,  1.19it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings: 100%|██████████| 8/8 [00:06<00:00,  1.24it/s]


  Language  Precision    Recall  F1_score  Accuracy
0      afr   0.275581  0.125585  0.151576  0.866122



<ipython-input-68-a88114e91a2a>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)
